## Before your start:

- Read the README.md file
- Follow each step as described in the instructions and take notes of the issues you find along the way
- Happy learning!

In [1]:
# Start by importing the table and the necessary libraries.

import pandas as pd
import numpy as np
import re

# Data Cleaning Code-Along

In this notebook we will be working on [Shark Attack](https://www.kaggle.com/teajay/global-shark-attacks/version/1) - a table of shark attack incidents compiled by the Global Shark Attack File. You will need to use your data wrangling skills to clean it up, prepare it to be analyzed, and then export it as a clean CSV data file.

## Step 1 - `INSPECTION`


### 1.1 Collect the data
- Read csv file into a pandas dataframe
- Take a look at the its **shape** and **features** by checking the first few rows and check the total missing values 

In [2]:
sharks = pd.read_csv('sharks_original.csv', engine='python')

In [3]:
sharks.shape

(5992, 24)

In [4]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

### 1.2 Take a look at the shape and features of the DataFrame 

Now that we now about its content, in order to get an idea of the table, we can print a sample of it!

**Tip**: Try setting no limit for the maximum number of rows and columns displayed when a frame is printed

In [5]:
pd.options.display.max_columns = None 

In [6]:
sharks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


Examine the dataframe for potential issues: `unexpected`, `incorrect`, or `inconsistent` 

- `Unexpected`: href formula, Unnamed 23, Unnamed 23
- `Incorrect`: 'Name': 'male'
- `Inconsistent`: 'Species': '2 m shark'


Some examples of issues are:
- Data entry errors in multiple columns and/or column names
- Few columns with irrelevant information 
- Columns with many different values 
- Columns with NaN values


### 1.3 Check for null values across all columns in the table

The columns with the highest amount of null values are the ones we should tackle first. Try to understand if they can be useful if cleaned.

In [7]:
null_sum = sharks.isnull().sum()
null_sum

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [8]:
def percent_NA(data):
    
    '''
    Returns a new dataframe including the total number of NA values and 
    the percentage of NA values in each column.
    '''
    
    null_sum = data.isnull().sum() 
    total = null_sum.sort_values(ascending=False)
    percent = (((null_sum / len(data.index))*100)).sort_values(ascending=False)
    
    # create the dataframe by concatenating through the columns 
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent of NA'])
    
    return df_NA

In [9]:
percent_NA(sharks)

,Number of NA,Percent of NA
Unnamed: 22,5991,99.983311
Unnamed: 23,5990,99.966622
Time,3213,53.621495
Species,2934,48.965287
Age,2681,44.742991
Sex,567,9.462617
Activity,527,8.795060
Location,496,8.277704
Area,402,6.708945
Name,200,3.337784


## Step 2 - `CLEANING`

Now that we have a better understanding of the data set, we can begin organising it and make it more understandable. 

### 2.1 Begin by tackling the **column names**. 

In [10]:
sharks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [11]:
def clean_col(data):
    
    '''
    Fixes all the messy column names by removing dots, whites spaces and using lower case
    '''

    col_clean = []
    
    for col in data.columns:
        col = col.strip().lower()
        col = col.replace('.',' ')
        col_clean.append(col)
        
    data.columns = col_clean
    
    return data.columns

In [12]:
clean_col(sharks)

Index(['case number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal (y/n)', 'time',
       'species', 'investigator or source', 'pdf', 'href formula', 'href',
       'case number 1', 'case number 2', 'original order', 'unnamed: 22',
       'unnamed: 23'],
      dtype='object')

### 2.2 Delete `unexpected` rows and columns that don’t fit your analysis 

In [13]:
to_drop = [  'case number',
             'area',
             'location',
             'name',
             'injury',
             'time',
             'species',
             'investigator or source',
             'pdf',
             'href formula',
             'href',
             'case number 1',
             'case number 2', 
             'original order', 
             'unnamed: 22', 
             'unnamed: 23']

sharks = sharks.drop(to_drop, axis=1)

In [14]:
sharks.shape

(5992, 8)

### 2.3 Check for `Incorrect` and `Inconsistent` values in the table

Use value_counts() to get a glimpse of some of the issues with each column

In [15]:
for i in sharks.columns:
    column_check = sharks[i].value_counts()
    print(column_check)   

1957         11
1942          9
1956          8
1950          7
1958          7
             ..
05-Sep-00     1
28-Feb-78     1
09-Feb-27     1
02-Jun-95     1
03-May-16     1
Name: date, Length: 5128, dtype: int64
2015    139
2011    128
2014    125
0       124
2013    122
       ... 
1742      1
1758      1
1818      1
1822      1
1595      1
Name: year, Length: 232, dtype: int64
Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: type, dtype: int64
USA                         2116
AUSTRALIA                   1279
SOUTH AFRICA                 565
PAPUA NEW GUINEA             133
NEW ZEALAND                  125
                            ... 
ARGENTINA                      1
JAVA                           1
MID-PACIFC OCEAN               1
SLOVENIA                       1
Between PORTUGAL & INDIA       1
Name: country, Length: 203, dtype: int64
Surfing                                                     

A few examples of issues that can be found in the results are:

- **date**: different lenghts and types of date formats
- **year**: different lengths (<4 digits) and value 0 
- **type**: inconsistent with Provoked/Unprovoked
- **country**: inconsistent with name of the country
- **activity**: unexpected large strings instead of categories
- **sex**: unexpected values besides M/F
- **age**: unexpected values besides 2 or max 3 digits
- **fatal**: inconsistent values with boolean logic (Y/N)

### 2.4 Deep Cleaning Time

It is now time to start tackling each issue individually and update the results in our DF.


- 2.4.1 Collect data from **DATE** to analyse **attacks per month**:

In [25]:
# Creating a new month column and inserting in the dataframe. 

col_month = []

for row in sharks['date']:
    month_row = ''.join(re.findall('\-[A-Za-z]{3}\-',row)).lower()
    month_row = re.sub('\-','',month_row)
        
        
# If month doesn't exist, the field will be filled with NaN.
        
    if month_row == '':
        month_row = np.nan

    col_month.append(month_row)
    

In [26]:
# Adding column MONTH and changing column order so it shows up next to date

sharks['month'] = col_month

column_order = ['date','month', 'year', 'type', 'country', 'activity', 'sex', 'age', 'fatal (y/n)']

sharks = sharks[column_order]

# Check how the new column looks and what's missing to be cleaned

sharks['month'].value_counts(dropna=False) 

NaN       870
jul       590
aug       537
sep       491
jan       476
jun       453
dec       397
oct       385
apr       375
mar       367
nov       365
may       344
feb       335
marmar      2
sepsep      1
novnov      1
janjan      1
jut         1
augaug      1
Name: month, dtype: int64

In [27]:
# Drop NaN values from the month column

sharks.dropna(subset=['month'], inplace=True)

In [28]:
# Correct the strings that contain the duplicated error by only considering the 3 first letters

for row in sharks['month']:
    if len(row) > 3:
        sharks['month'].replace(row,row[:3], inplace=True)
    elif row == 'jut':
        sharks['month'].replace(row,'jun', inplace=True)

In [29]:
pd.unique(sharks['month'])

array(['sep', 'aug', 'jul', 'jun', 'may', 'apr', 'mar', 'feb', 'jan',
       'dec', 'nov', 'oct'], dtype=object)

- 2.4.2 Clean the column **SEX** to analyse **distribution of attacks per gender**:

In [30]:
pd.unique(sharks['sex'])

array(['M', nan, 'F', 'M ', 'lli', 'N', '.'], dtype=object)

In [31]:
# This function will take all the values and check if they are M or F, otherwise replace as NaN

def sex_clean(sex):

    if sex == 'M' or sex == 'F':
        return sex
    else:
        return np.nan  

In [32]:
sharks['sex'] = sharks['sex'].apply(sex_clean)
sharks['sex'].value_counts(dropna=False) 

M      4164
F       525
NaN     433
Name: sex, dtype: int64

- 2.4.3 Clean the column **AGE** to analyse **distribution of attacks per age**:

In [33]:
# This function will take all the values and replace as 0 if they are not integers

def age_clean(age):
    try:
        age = int(age)
    except ValueError:
        age = 0

# Since there are also ranges in AGE, this condition will replace them for NaN values
        
    if (age > 0 and age <= 100):
        return age
    else:
        return np.nan    

In [34]:
# Create a new DF to analyse the AGE data  

age_dF = sharks['age'].apply(age_clean)

- 2.4.4 Clean the column **FATAL** to analyse **fatality of the attacks**:

In [35]:
# This function will take all the values and check if Y or N, otherwise replace with NaN

def fatal_clean(fatal):
    if fatal == 'N' or fatal == 'Y':
        return fatal
    else:
        return np.nan

In [36]:
sharks['fatal (y/n)'] = sharks['fatal (y/n)'].apply(fatal_clean)   
sharks['fatal (y/n)'].value_counts(dropna=False) 

N      3784
Y      1256
NaN      82
Name: fatal (y/n), dtype: int64

## Step 3 - `Verifying`

After having organised and cleaned the data set, the last step is to inspect the results to verify correctness. 


In [37]:
sharks.shape

(5122, 9)

In [38]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5122 entries, 0 to 5980
Data columns (total 9 columns):
date           5122 non-null object
month          5122 non-null object
year           5122 non-null int64
type           5122 non-null object
country        5097 non-null object
activity       4751 non-null object
sex            4689 non-null object
age            3139 non-null object
fatal (y/n)    5040 non-null object
dtypes: int64(1), object(8)
memory usage: 400.2+ KB


In [39]:
# Check for potential anomalies in the clean data set

sharks.head()

,date,month,year,type,country,activity,sex,age,fatal (y/n)
0,18-Sep-16,sep,2016,Unprovoked,USA,Surfing,M,16,N
1,18-Sep-16,sep,2016,Unprovoked,USA,Surfing,M,36,N
2,18-Sep-16,sep,2016,Unprovoked,USA,Surfing,M,43,N
3,17-Sep-16,sep,2016,Unprovoked,AUSTRALIA,Surfing,M,NaN,N
4,16-Sep-16,sep,2016,Unprovoked,AUSTRALIA,Surfing,M,NaN,N


In [40]:
sharks['year'].value_counts(dropna=False)

2015    138
2011    127
2014    122
2013    122
2009    119
       ... 
1787      1
1807      1
1819      1
1827      1
1742      1
Name: year, Length: 200, dtype: int64

In [41]:
sharks.dtypes

date           object
month          object
year            int64
type           object
country        object
activity       object
sex            object
age            object
fatal (y/n)    object
dtype: object

- Final step is to export the clean dataset as a csv file so it can be thoroughly analysed 

In [42]:
sharks.to_csv("./sharks_clean.csv", index = False)